In [11]:
import torch
from torch import nn
import torchvision
from PIL import Image

device = torch.device('mps')

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=2)

# for batch, labels in train_loader:
#     print(batch)
#     print(batch.shape)
#     print(labels.shape)
#     break

In [13]:
from vision_transformer import VisionTransformer

device = torch.device('mps')

model = VisionTransformer().to(device)

x = torch.randn(size=(2, 3, 32, 32), device=device)

out = model(x)

print(out)
print(out.shape)

tensor([[[-1.4266,  0.3027, -0.1247,  ...,  0.4989, -0.5668,  0.0913],
         [-0.0986, -0.9967,  0.0490,  ..., -0.6829,  0.5010,  0.5508],
         [-0.5173,  0.1241,  0.2177,  ...,  0.0602, -0.3026, -0.1654],
         ...,
         [ 0.1143, -0.4601,  1.2222,  ...,  0.3347,  0.0179,  0.5887],
         [ 1.1621,  0.0930, -0.1063,  ..., -1.6230, -0.0873,  0.7128],
         [-0.0431, -0.7173, -0.7456,  ...,  0.1139,  0.3585, -0.0726]],

        [[-1.4269,  0.3029, -0.1237,  ...,  0.4985, -0.5678,  0.0920],
         [-0.0379, -0.5832,  0.3401,  ..., -0.5891,  0.7941,  0.6414],
         [-0.7758,  0.3890,  0.3443,  ..., -0.7045, -0.5965, -1.0891],
         ...,
         [-0.1367, -0.2118,  1.1660,  ...,  0.6019, -0.3521,  0.6427],
         [ 0.7963, -0.2981, -0.0935,  ..., -1.7572, -0.0958,  1.0378],
         [-0.4049, -0.5793, -0.7363,  ..., -0.0612, -0.5588, -0.0694]]],
       device='mps:0', grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 65, 512])
